<a href="https://colab.research.google.com/github/nahumsa/Reinforcement-Learning/blob/master/Vanilla%20Policy%20Gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vanilla Policy Gradient

In this notebook I will implement on Tensoflow 2 an implementation of the [Vanilla Policy Gradient](https://spinningup.openai.com/en/latest/algorithms/vpg.html) as it is explained on the [Spinning Up](https://spinningup.openai.com/en/latest/index.html) by Open AI, which is an incredible resource for Reinforcement Learning.

# 0) Importing dependencies

In [0]:
!apt-get install -y xvfb python-opengl x11-utils > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

import tensorflow as tf

import numpy as np
import base64, io, time, gym
import IPython, functools
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1) Policy Gradient

Firstly, we need to define how to efficiently calculate the Policy Gradient, this is done by updating the expected return: 
$$
J(\pi_\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \big[ R(\tau) \big]
$$ 

## 1.1) Return Function

There are severous return functions. In this section I will show some implmentations of them.

### 1.1.1) Finite-horizon undiscounted return
This return is defined by: 

$$
R(\tau) = \sum_{t=0}^{\infty} r_t
$$


In [0]:
def normalized(x): 
  x -= np.mean(x)
  x /= np.std(x)
  return x.astype(np.float32)

def undiscount_rewards(rewards):
  discounted_rewards = np.zeros_like(rewards)
  R = 0
  # Do a backward update on rewards
  for t in reversed(range(0,len(rewards))):
    R = R + rewards[t]
    undiscounted_rewards[t] = R
  
  return normalized(undiscounted_rewards)

### 1.1.2) Finite-horizon discounted return
This return is defined by: 

$$
R(\tau) = \sum_{t=0}^{\infty} \gamma^t r_t
$$


In [0]:
def discount_rewards(rewards, gamma=0.99):
  discounted_rewards = np.zeros_like(rewards)
  R = 0
  # Do a backward update on rewards
  for t in reversed(range(0,len(rewards))):
    R = R*gamma + rewards[t]
    discounted_rewards[t] = R
  
  return normalized(discounted_rewards)

# 1.2) Optimizing the Policy

We would like to optimize the policy by gradient ascent:

$$
\theta_{k+1} = \theta_{k} + \alpha \nabla_\theta J(\pi_\theta) |_{\theta_{k}}
$$

We define the policy gradient as the gradient of the policy performance ($\nabla_\theta J(\pi_\theta)$) and algorithms that optimize the policy are called **policy gradient algorithms**.

We need a expression that makes it easier to calculade the policy gradient, this can be done by the following:

$$
\nabla_\theta J(\pi_\theta) = \nabla_\theta \mathbb{E}_{\tau \sim \pi_\theta} \big[ R(\tau) \big] = \nabla_\theta \int_{\tau} P(\tau | \theta) R(\tau) =  \int_{\tau} \big( \nabla_\theta P(\tau | \theta) \big) R(\tau)
$$

For calculating the term in parenthesis we use the log-derivative trick, which combines the derivative of the log of a function and the chain rule:

$$
\nabla_\theta  \mathrm{log}P(\tau | \theta) = \frac{\nabla_\theta P(\tau | \theta)}{P(\tau | \theta)} \Rightarrow \nabla_\theta  P(\tau | \theta) = P(\tau | \theta)\nabla_\theta  \mathrm{log}P(\tau | \theta)
$$

Then we have:

$$
\nabla_\theta J(\pi_\theta) = \int_{\tau} \big( P(\tau | \theta)\nabla_\theta  \mathrm{log}P(\tau | \theta) \big) R(\tau) = \mathbb{E}_{\tau \sim \pi_\theta} \bigg[ \nabla_\theta  \mathrm{log}P(\tau | \theta) R(\tau) \bigg]
$$

To get a final expression, we need to see the probability of a trajectory given that actions come fom $\pi_\theta$ is:

$$
P(\tau|\theta) = \rho(s_0) \prod_{t=0}^{T} P(s_{t+1}|s_t, a_t) \pi_\theta (a_t|s_t)
$$

Then the log of that probability is:

$$
\mathrm{log} P(\tau|\theta) = \rho(s_0) +  \sum_{t=0}^{T} \bigg( \mathrm{log}P(s_{t+1}|s_t, a_t) + \mathrm{log} \pi_\theta (a_t|s_t) \bigg)
$$

Then we have the policy gradient, noting that the only term that doesn't have gradient 0 is the term with $\pi_\theta$:

$$
\nabla_\theta J(\pi_\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \bigg[ \sum_{t=0}^T \nabla_\theta \mathrm{log} \pi_\theta(a_t|s_t)  R(\tau) \bigg]
$$

Since this is an expectation value we can estimate it by sampling the mean of collected trajectories, D:

$$ 
\hat{g} = \frac{1}{|D|} \sum_{\tau \in D} \sum_{t=0}^T \nabla_\theta \mathrm{log} \pi_\theta(a_t|s_t)  R(\tau)
$$

Where $|D|$ is the number of trajectories in D.

In [0]:
class Memory:
  """ Memory used in training in order to keep each 
  Observation, Actions and Rewards.
  """
  
  def __init__(self):
    self.clear()
  
  def clear(self):
    self.observations = []
    self.actions = []
    self.rewards = []

  def add_to_memory(self, new_observation, new_action, new_reward):
    self.observations.append(new_observation)
    self.actions.append(new_action)
    self.rewards.append(new_reward)

memory = Memory()

In [0]:
def choose_action(model, observation):
  """Choosing an action from a model.
  """ 
  observation = np.expand_dims(observation, axis=0)
  logits = model.predict(observation)
  
  #Policy
  prob_weights = tf.nn.softmax(logits).numpy()
  
  #Chosing an action according to the policy
  action = np.random.choice(n_actions, size=1, p=prob_weights.flatten())[0]  
  return action

In [0]:
def compute_loss(logits, actions, rewards):
  """Loss function for the Vanilla Policy Gradient
  """
  neg_logprob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=actions)
  loss = tf.reduce_mean(rewards*neg_logprob)
  return loss

In [0]:
def train_step(model, optimizer, observations, actions, discounted_rewards):
  """Gradient ascend step for the model
  """
  
  with tf.GradientTape() as tape:
    #Forwardprop
    logits = model(observations)
    loss = compute_loss(logits,actions,discounted_rewards)
  
  #Backprop
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads,model.trainable_variables))

In [0]:
epochs = 5000
epochs = range(epochs)

def training_loop()
   
  observation = env.reset()
  memory.clear()

  while True:
    action = choose_action(car_model, observation)
    next_observation, reward, done, info = env.step(action)

    memory.add_to_memory(observation, action, reward)

    if done:
      total_reward = sum(memory.rewards)
      smoothed_reward.append(total_reward)

      train_step(car_model, optmizer,
                observations= np.vstack(memory.observations),
                actions=np.array(memory.actions),
                discounted_rewards= discount_rewards(memory.rewards)
                )
      memory.clear()
      break
    
    observation = next_observation

# 2) Resources

[Introduction to Policy Optimization](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html)

[Vanilla Policy Gradient](https://spinningup.openai.com/en/latest/algorithms/vpg.html)

Schulman et al. - [High-Dimensional Continuous Control Using Generalized Advantage Estimation](https://arxiv.org/abs/1506.02438)

Sutton et al - [Policy Gradient Methods for Reinforcement Learning with Function Approximation](https://papers.nips.cc/paper/1713-policy-gradient-methods-for-reinforcement-learning-with-function-approximation.pdf)